In [2]:
import duckdb

In [3]:
# read csv file

with duckdb.connect(database='data.db', read_only=False) as con:
    con.execute("create or replace table wifi_sessions as select * from 'data/hackacity_wifi_data.csv'")

In [ ]:
with duckdb.connect(database='data.db', read_only=False) as con:
    con.execute("""INSTALL spatial;
LOAD spatial;""")
    con.execute("create or replace table access_pointss as select * from st_read('data/aps_hackacity.xlsx', open_options = ['HEADERS=FORCE'])")

IOException: IO Error: Could not set lock on file "/Users/jose.cabeda/Git/hackacity/data.db": Conflicting lock is held in /opt/homebrew/Cellar/duckdb/1.1.3/bin/duckdb (PID 77675) by user jose.cabeda. See also https://duckdb.org/docs/connect/concurrency

In [22]:
with duckdb.connect(database='./analysis/sources/wifi/data.db', read_only=False) as con:
   data = con.execute("select * from wifi_sessions limit 100").fetch_df()

data

,column0,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,0,fbfe95a31b975faa4b79968f95d834f2204ff050c4cb17...,2024-09-18 14:00:00,0,0,0.0,412a9bf588f4a609cd8e4c8fc8b69482d6f69079f274de...,e0c8b4c72e48ff3726f36a987f29bab9b3d718646b3a6e...,eduroam
1,1,7330b1f18b414f72c4895700952fb9f2f5aa4cd3c15369...,2024-09-18 13:00:00,0,0,0.0,d5f9cf3d6db078eec48f1f235072ddcec82917cf70aca7...,7b85b2daca64da5d39240d583c6097589a9a5dab4215b0...,eduroam
2,2,8efbb33c1a4b95f434e75cde462caea6dfdf964631744f...,2024-09-12 07:00:00,0,0,6.0,a1b5475191cbe73bff8f88915727f64975a0fd3927337f...,bac16ce82d2706d6346e3929a1a46b4fa8c23af51daa82...,Porto. Free Wi-Fi
3,3,429785416485c487d271060e329442b10e1b965621dc17...,2024-09-12 01:00:00,2,30,32.0,67004872f49151a450a1a06a7b2e9759f247cc87c718fb...,a624b94f6637ac8e51fab1d1d6bc1eca5c8949e4e57783...,eduroam
4,4,147932564d4a91bdf901570a3f379158e0efe674ac55d4...,2024-09-12 02:00:00,0,1,6.0,8fead2e516c0315a13d8a826ee4f0c1e74a256d689a895...,610f3720cc37d9195ab25e780f36690113624f19039fec...,eduroam
...,...,...,...,...,...,...,...,...,...
95,95,cf36fe4b3f7928fa373a4595ba57b10444c4821b02b7d3...,2024-09-12 01:00:00,1,0,10.0,927649f7c57c40d61584bd801eed11e2a624f108df6073...,c5b75ca3821a19ebbaa99bfd055b16b36ef85c479c5287...,eduroam
96,96,496392e649d9cd03555f33025f4210c683e81005089328...,2024-09-12 05:00:00,0,0,6.0,43ed05818d69034a7b5b48e74ceaa939ce3f2a935c5d69...,7501b9e9437bf71fc10c363fcbe3fc6c36c5f447f93d22...,Porto. Free Wi-Fi
97,97,97fa7afd9ba1e853c95aaa88112db37c4d8b7f43bd8a0d...,2024-09-12 01:00:00,0,0,1.0,aff5b7218430106765dad78c04de81fdbb3dce02e7c78b...,4cad16c07e8c4d222c35d87e0e91ca7f748373f1733a74...,Porto. Free Wi-Fi
98,98,0caa602db0260f075038208f2e680fa557933c4e1ed274...,2024-09-12 01:00:00,0,0,5.0,be12ba143d676b997156684de448b597e3afb3cb2a3ca6...,000a9b5f3e9924b838334ddc856fb44e865abff86f6ef2...,eduroam


In [ ]:
wifi_sessions

In [ ]:
CREATE TABLE wifi_sessions (
    acctsessionid VARCHAR,
    acctstarttime TIMESTAMP,
    acctsessiontime INTEGER,
    upload INTEGER,
    download INTEGER,
    calledstationid VARCHAR,
    callingstationid VARCHAR,
    calledstation_ssid VARCHAR
);